In [1]:
%config IPCompleter.greedy=True
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import time
import re
import scipy
from datetime import time, timedelta
import seaborn as sns
pd.set_option('display.max_columns', 100)

from dateutil.parser import parse

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from pandas.api.types import CategoricalDtype

In [4]:
def trans_time(t):
    if t == 'None':
        return np.datetime64('NaT')
    elif pd.isna(t):
        return np.datetime64('NaT')
    else:
        return np.datetime64(t)

In [5]:
def trans_float(df):
    for i in df.columns:
        if df[i].dtype == object:
            if ((df[i]== 'None')|(df[i]== 'Null')).any():
                df[i].replace('None',np.nan,inplace = True)
                df[i].replace('Null',np.nan,inplace = True)
                df[i] = df[i].astype(float)
            else:
                df[i] = df[i].astype(float)

In [6]:
def trans_time(t):
    if t == 'None':
        return np.datetime64('NaT')
    elif pd.isna(t):
        return np.datetime64('NaT')
    else:
        return parse(t)

In [2]:
def read_largefile(path, encoding):
    test = pd.read_csv(path, sep=',',encoding = encoding,nrows= 10000)
    nan_count_list = []
    col_sel_list = []
    for i in range(len(test.columns.to_list())):
        nan_count_list.append((pd.isna(test.iloc[:,i]).sum()/10000)*100)

    for i in range(len(nan_count_list)):
        if nan_count_list[i]<=50:
            col_sel_list.append(test.columns.to_list()[i])
    
    reader = pd.read_csv(path,usecols= col_sel_list, sep=',', iterator=True,encoding = encoding)
    loop = True
    chunkSize = 100000
    chunks = []
    while loop:
        try:
            chunk = reader.get_chunk(chunkSize)
            chunks.append(chunk)
        except StopIteration:
            loop = False
            print("Iteration is stopped.")
    df = pd.concat(chunks, ignore_index=True)
    return df

# 生命体征

In [8]:
vitals = read_largefile('../Local_hosp_data/vitals.csv', encoding = 'gb2312')

Iteration is stopped.


In [9]:
vitals.VITAL_SIGNS.value_counts()

脉搏        7531181
呼吸        7441957
血压high    5205383
血压Low     5205149
体温        4782498
血氧        3293086
血糖        1616049
尿量         701005
总入量        677114
心率         202266
Name: VITAL_SIGNS, dtype: int64

In [10]:
vitals = vitals[vitals.VITAL_SIGNS.isin(['脉搏','呼吸','血压high','血压Low','血氧','体温','血糖','心率','尿量','总入量','总出量','呼吸机辅助'])].\
drop_duplicates().dropna(axis = 0,subset = ['VITAL_SIGNS_VALUES'])

In [11]:
#vitals.to_csv('vitals.csv', index = False)

# 实验室检查

In [3]:
lab = read_largefile('../Local_hosp_data/lab.csv', encoding = 'gb18030')

Iteration is stopped.


In [5]:
['ALP', 'ALT', 'APPT', 'AST','BNP', 'CRP', 'D_dimer', 'IL_6', 'INR', 'LYN', 'PA', 'PCT', 'PT', 'PTT',
'albumin', 'amylase', 'bands', 'basophil', 'bilirubin',
'bilirubin_direct', 'ca', 'chloride', 'ck', 'ck_mb', 'creatinine',
'eosinophil', 'fibrinogen', 'gamma_GT', 'glucose', 'hct_pcv',
'hemoglobin', 'lipase', 'mb', 'mch', 'mchc', 'mcv', 'mg', 'monocytes',
'mpv', 'p', 'platelet', 'potassium', 'rbc', 'rdw', 'sodium', 'urea','wbc']

['ALP',
 'ALT',
 'APPT',
 'AST',
 'BNP',
 'CRP',
 'D_dimer',
 'IL_6',
 'INR',
 'LYN',
 'PA',
 'PCT',
 'PT',
 'PTT',
 'albumin',
 'amylase',
 'bands',
 'basophil',
 'bilirubin',
 'bilirubin_direct',
 'ca',
 'chloride',
 'ck',
 'ck_mb',
 'creatinine',
 'eosinophil',
 'fibrinogen',
 'gamma_GT',
 'glucose',
 'hct_pcv',
 'hemoglobin',
 'lipase',
 'mb',
 'mch',
 'mchc',
 'mcv',
 'mg',
 'monocytes',
 'mpv',
 'p',
 'platelet',
 'potassium',
 'rbc',
 'rdw',
 'sodium',
 'urea',
 'wbc']

In [ ]:
#mimic 
# mg,50960,mg/dl; mmol/L
# phosphate,50970,mg/dl; mmol/L
# 'rbc', 51279, m/uL; 10^12/L
#  'platelet',51265, K/uL; 10^9/L
#  'rdw',51277,%; %
#  'chloride',50902,mEq/L; mmol/L
#  'sodium',50983,mEq/L; mmol/L
#  'hemoglobin',51222,g/dL; g/L

In [17]:
lab[(lab['LABEL'] == 'p')].UNITS.value_counts()

mmol/L    456144
Name: UNITS, dtype: int64

# icustay_detail

In [19]:
final_cohort = pd.read_csv('final_cohort.csv')

In [20]:
final_cohort = final_cohort[final_cohort['LOS_ICU']>0]

In [21]:
for i in ['ADMISSION_DATE_TIME','DISCHARGE_DATE_TIME','TF_ADMISSION_DATE_TIME', 'TF_DISCHARGE_DATE_TIME']:
    final_cohort[i] = final_cohort[i].apply(trans_time)

# 实验室检查值

In [49]:
lab_sel = lab[lab['PATIENT_ID'].isin(final_cohort['PATIENT_ID'].to_list())]

In [50]:
lab_bc = pd.merge(final_cohort[['PATIENT_ID','TF_ADMISSION_DATE_TIME']],lab_sel,how = 'left', on = ['PATIENT_ID'])

In [51]:
lab_bc[(lab_bc['LABEL'] == 'wbc')].PRINT_CONTEXT.value_counts()

3.5～10           82750
3.5～10×10^9/L    68664
3.5-10           48777
Name: PRINT_CONTEXT, dtype: int64

In [52]:
lab_bc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11428788 entries, 0 to 11428787
Data columns (total 12 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   PATIENT_ID               object        
 1   TF_ADMISSION_DATE_TIME   datetime64[ns]
 2   VISIT_ID                 float64       
 3   TEST_NO                  int64         
 4   ITEM_NO                  int64         
 5   REQUESTED_DATE_TIME      object        
 6   SPCM_RECEIVED_DATE_TIME  object        
 7   LABEL                    object        
 8   RESULT                   object        
 9   UNITS                    object        
 10  RESULTS_RPT_DATE_TIME    object        
 11  PRINT_CONTEXT            object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(8)
memory usage: 1.1+ GB


In [53]:
for i in ['REQUESTED_DATE_TIME','RESULTS_RPT_DATE_TIME']:
    lab_bc[i] = lab_bc[i].apply(trans_time)

In [54]:
lab_bc_1stday = lab_bc[((lab_bc['REQUESTED_DATE_TIME'] - lab_bc['TF_ADMISSION_DATE_TIME']).dt.total_seconds()/3600<=48)&\
                       ((lab_bc['REQUESTED_DATE_TIME'] - lab_bc['TF_ADMISSION_DATE_TIME']).dt.total_seconds()/3600>=-24)]

In [55]:
lab_bc_1stday.dropna(axis = 0, subset = ['RESULT'], inplace = True)

In [56]:
lab_bc_1stday = lab_bc_1stday.drop_duplicates()

In [57]:
def sub_f(x):
    if x == 'nan':
        return np.nan
    else:
        x_togroup = re.search("\d+(\.\d+)?",x)
        if x_togroup == None:
            return np.nan
        else:
            return x_togroup.group()


lab_bc_1stday['RESULT'] = lab_bc_1stday['RESULT'].astype(str).apply(sub_f).astype(float)

In [163]:
lab_bc_1stday[(lab_bc_1stday['LABEL'] == 'APPT')].PRINT_CONTEXT.value_counts()

30～45s    30175
Name: PRINT_CONTEXT, dtype: int64

In [59]:
def blood_def_clean(df):
    if df['LABEL'] in ['bands','LYN','basophil','monocytes','eosinophil'] and (df['RESULT']>=1 or df['RESULT']<0):
        return np.nan    
    else:
        return df['RESULT']

In [60]:
lab_bc_1stday['RESULT'] = lab_bc_1stday.apply(blood_def_clean,axis = 1)

In [61]:
lab_count = lab_bc_1stday.LABEL.value_counts()

In [62]:
lab_bc_sel = lab_bc_1stday[lab_bc_1stday.LABEL.isin(lab_count.head(47).index.to_list())]

In [63]:
lab_bc_sel = lab_bc_sel[['PATIENT_ID', 'TF_ADMISSION_DATE_TIME', 
       'REQUESTED_DATE_TIME',  'LABEL','RESULT']].drop_duplicates().dropna()

In [64]:
lab_bc_sel_1st = lab_bc_sel.groupby(['PATIENT_ID','TF_ADMISSION_DATE_TIME','LABEL']).apply(lambda x: x.sort_values(['REQUESTED_DATE_TIME'],ascending = True).head(1))

In [65]:
lab_1st = lab_bc_sel_1st.reset_index(drop = True)

In [66]:
lab_1st.drop(columns = ['REQUESTED_DATE_TIME'],inplace = True)

In [67]:
#lab_1st[lab_1st['PATIENT_ID'] =='S118067120'].set_index(['PATIENT_ID','TF_ADMISSION_DATE_TIME','LABEL']).unstack()

In [68]:
lab_1st = lab_1st.set_index(['PATIENT_ID','TF_ADMISSION_DATE_TIME','LABEL']).unstack()

In [69]:
lab_name = []
for i in lab_1st.reset_index().columns.to_list()[2:]:
    lab_name.append(i[1])

In [70]:
lab_name.insert(0,lab_1st.reset_index().columns.to_list()[0][0])
lab_name.insert(1,lab_1st.reset_index().columns.to_list()[1][0])

In [71]:
lab_1st_re = lab_1st.reset_index()

In [72]:
lab_1st_re.columns = lab_name

In [73]:
#lab_1st_re[lab_1st_re['C-反应蛋白测定'].str.contains(r'[^((0-9)|\.)]')]

In [74]:
lab_1st_re_re = lab_1st_re.copy()

In [75]:
lab_1st_re_re.columns

Index(['PATIENT_ID', 'TF_ADMISSION_DATE_TIME', 'ALP', 'ALT', 'APPT', 'AST',
       'BNP', 'CRP', 'D_dimer', 'IL_6', 'INR', 'LYN', 'PA', 'PCT', 'PT', 'PTT',
       'albumin', 'amylase', 'bands', 'basophil', 'bilirubin',
       'bilirubin_direct', 'ca', 'chloride', 'ck', 'ck_mb', 'creatinine',
       'eosinophil', 'fibrinogen', 'gamma_GT', 'glucose', 'hct_pcv',
       'hemoglobin', 'lipase', 'mb', 'mch', 'mchc', 'mcv', 'mg', 'monocytes',
       'mpv', 'p', 'platelet', 'potassium', 'rbc', 'rdw', 'sodium', 'urea',
       'wbc'],
      dtype='object')

In [76]:
lab_ = pd.merge(final_cohort[['PATIENT_ID','TF_ADMISSION_DATE_TIME']],lab_1st_re_re, how = 'left', on = ['PATIENT_ID','TF_ADMISSION_DATE_TIME'])

In [77]:
lab_ = lab_.drop_duplicates()

In [78]:
def lookup_miss(df):
    nadf = pd.DataFrame(columns=['nan_nums','nan_ratios'])
    for i in df.columns:
        nadf.loc[i,'nan_nums'] = pd.isna(df[i]).sum()
        nadf.loc[i,'nan_ratios'] = (nadf.loc[i,'nan_nums']/df.shape[0])*100
    return nadf.sort_values(by = 'nan_ratios', ascending = False)

In [79]:
lookup_miss(lab_)

,nan_nums,nan_ratios
IL_6,5560,41.594973
PCT,4809,35.976659
BNP,2805,20.984514
lipase,2738,20.48328
amylase,2685,20.086781
D_dimer,2601,19.458368
ALP,2212,16.548216
gamma_GT,2188,16.368669
mb,2119,15.852473
p,2053,15.358719


# 生命体征

In [80]:
vitals.VITAL_SIGNS.value_counts()[0:20]

脉搏        7531180
呼吸        7441956
血压high    5205382
血压Low     5205148
体温        4782497
血氧        3293086
血糖        1616049
尿量         701005
总入量        677114
心率         202266
Name: VITAL_SIGNS, dtype: int64

In [81]:
vitals = vitals[vitals.VITAL_SIGNS.isin(['脉搏','呼吸','血压high','血压Low','血氧','体温','血糖','心率'])].\
drop_duplicates().dropna(axis = 0,subset = ['VITAL_SIGNS_VALUES'])

In [82]:
vitals_bc = pd.merge(final_cohort[['PATIENT_ID','VISIT_ID','TF_ADMISSION_DATE_TIME']],vitals,how = 'left', on = ['PATIENT_ID','VISIT_ID'])

In [83]:
vitals_bc['TIME_POINT'] = vitals_bc['TIME_POINT'].apply(trans_time)

In [84]:
vitals_bc_1stday = vitals_bc[((vitals_bc['TIME_POINT'] - vitals_bc['TF_ADMISSION_DATE_TIME']).dt.total_seconds()/3600>=-24)&\
((vitals_bc['TIME_POINT'] - vitals_bc['TF_ADMISSION_DATE_TIME']).dt.total_seconds()/3600<=24)]

In [85]:
vitals_bc_1stday.columns

Index(['PATIENT_ID', 'VISIT_ID', 'TF_ADMISSION_DATE_TIME', 'RECORDING_DATE',
       'TIME_POINT', 'VITAL_SIGNS', 'VITAL_SIGNS_VALUES', 'UNITS'],
      dtype='object')

In [86]:
vitals_bc_1stday = vitals_bc_1stday[['PATIENT_ID', 'TF_ADMISSION_DATE_TIME', 'TIME_POINT', 'VITAL_SIGNS', 'VITAL_SIGNS_VALUES']]

In [87]:
vitals_bc_1stday_1st = vitals_bc_1stday.groupby(['PATIENT_ID','TF_ADMISSION_DATE_TIME', 'VITAL_SIGNS']).apply(lambda x: x.sort_values(['TIME_POINT'],ascending = True).head(1))

In [88]:
vitals_bc_1stday_1st = vitals_bc_1stday_1st.reset_index(drop = True)

In [89]:
vitals_1st = vitals_bc_1stday_1st.drop(columns = ['TIME_POINT']).set_index(['PATIENT_ID','TF_ADMISSION_DATE_TIME','VITAL_SIGNS']).unstack()

In [90]:
vitals_name = []
for i in vitals_1st.reset_index().columns.to_list()[2:]:
    vitals_name.append(i[1])

In [91]:
vitals_name.insert(0,vitals_1st.reset_index().columns.to_list()[0][0])
vitals_name.insert(1,vitals_1st.reset_index().columns.to_list()[1][0])

In [92]:
vitals_1st = vitals_1st.reset_index()

In [93]:
vitals_1st.columns = vitals_name

In [94]:
pd.isna(vitals_1st).sum()

PATIENT_ID                    0
TF_ADMISSION_DATE_TIME        0
体温                            0
呼吸                           28
心率                        12958
脉搏                            0
血压Low                         5
血压high                        5
血氧                         5762
血糖                         8408
dtype: int64

In [95]:
vitals_1st.drop(columns = ['心率','血糖'], inplace = True)

In [96]:
vitals_1st.columns

Index(['PATIENT_ID', 'TF_ADMISSION_DATE_TIME', '体温', '呼吸', '脉搏', '血压Low',
       '血压high', '血氧'],
      dtype='object')

In [97]:
vitals_1st.columns = ['PATIENT_ID', 'TF_ADMISSION_DATE_TIME', 'Temperature', 'RR', 'HR', 'DBP','SBP', 'SPO2']

In [98]:
bc_cohort = pd.merge(final_cohort,lab_,how = 'left', on = ['PATIENT_ID', 'TF_ADMISSION_DATE_TIME']).\
merge(vitals_1st, how = 'left', on = ['PATIENT_ID', 'TF_ADMISSION_DATE_TIME'])

In [99]:
bc_cohort['GENDER'] = bc_cohort['GENDER'].apply(lambda x: 1 if x == 'M' else 0)

In [100]:
bc_cohort = pd.get_dummies(data = bc_cohort, columns= ['ICU_TYPE'])

In [101]:
bc_cohort = bc_cohort.drop(columns = ['previous_mdrpos_lessthan7d','previous_mdrneg_lessthan7d','los_icu_exc','stay_chart_included'])

In [102]:
icustay_detail = pd.read_csv('../Local_hosp_data/icustay_detail.csv',encoding = 'gb18030')
icustay_detail = icustay_detail[~pd.isna(icustay_detail['ICU_TYPE'])]
icustay_detail['TF_ADMISSION_DATE_TIME'] = icustay_detail['TF_ADMISSION_DATE_TIME'].apply(trans_time)

icustay_detail['icustay_rank']= icustay_detail.groupby(['PATIENT_ID'])['TF_ADMISSION_DATE_TIME'].rank()

In [103]:
icustay_detail['icustay_rank'].value_counts()

1.0     67751
2.0     10772
3.0      2346
4.0       750
5.0       342
6.0       188
7.0       128
8.0        98
9.0        73
10.0       58
11.0       50
12.0       42
13.0       33
14.0       24
15.0       22
16.0       18
17.0       17
18.0       14
19.0        8
20.0        7
21.0        4
22.0        3
23.0        2
Name: icustay_rank, dtype: int64

In [104]:
bc_cohort.drop(columns = ['icustay_rank'], inplace = True)

In [105]:
bc_cohort = bc_cohort.merge(icustay_detail[['PATIENT_ID', 'TF_ADMISSION_DATE_TIME','icustay_rank']], how = 'left', on = ['PATIENT_ID', 'TF_ADMISSION_DATE_TIME'])

In [106]:
#住院次数
bc_cohort['hospstay_seq'] = bc_cohort['VISIT_ID']

#住院时长
bc_cohort['los_hospital'] = (bc_cohort['DISCHARGE_DATE_TIME'] - bc_cohort['ADMISSION_DATE_TIME']).dt.total_seconds()/(3600*24)

#死亡信息
mort = pd.read_csv('../Local_hosp_data/mort.csv')

mort['DOD_HOSP'] = mort['DOD_HOSP'].apply(trans_time)

bc_cohort = pd.merge(bc_cohort, mort, how = 'left', on = ['PATIENT_ID','VISIT_ID'])

bc_cohort['DOD'] = \
bc_cohort[['ADMISSION_DATE_TIME','DISCHARGE_DATE_TIME','DOD_HOSP']].\
apply(lambda x: 1 if ((x[2]-x[0]).total_seconds()>=0) and ((x[2]-x[1]).total_seconds()<=0) else 0, axis = 1)

In [107]:
bc_cohort = bc_cohort.drop_duplicates()

In [108]:
#bc_cohort_icu = bc_cohort[(bc_cohort['ICU_TYPE_ICU'] == 1)|(bc_cohort['ICU_TYPE_EICU'] == 1)]#.drop_duplicates()

In [109]:
bc_cohort_icu = bc_cohort[(bc_cohort['ICU_TYPE_ICU'] == 1)]

In [110]:
icustay_detail.columns

Index(['PATIENT_ID', 'VISIT_ID', 'GENDER', 'AGE', 'ADMISSION_DATE_TIME',
       'DISCHARGE_DATE_TIME', 'DATE_OF_BIRTH', 'ADMISSION_TYPE',
       'ADMISSION_CONDITION', 'DEPT_STAYED', 'ICU_TYPE', 'ICU_FLAG',
       'TF_ADMISSION_DATE_TIME', 'TF_DISCHARGE_DATE_TIME', 'LOS_ICU',
       'IDENTITY', 'OCCUPATION', 'MARITAL_STATUS', 'MAILING_ADDRESS',
       'BLOOD_TYPE', 'BLOOD_TYPE_RH', 'ALERGY_DRUGS', 'MR_VALUE', 'MR_QUALITY',
       'DISCHARGE_DISPOSITION', 'CHARGE_TYPE', 'MEDICAL_PAY_WAY',
       'TOTAL_COSTS', 'TOTAL_PAYMENTS', 'SERIOUS_COND_DAYS',
       'CRITICAL_COND_DAYS', 'ICU_DAYS', 'CCU_DAYS', 'BODY_HEIGHT',
       'BODY_WEIGHT', 'HBSAG_INDICATOR', 'HCV_AB_INDICATOR',
       'HIV_AB_INDICATOR', 'icustay_rank'],
      dtype='object')

In [111]:
hw = icustay_detail[['PATIENT_ID', 'VISIT_ID','BODY_HEIGHT','BODY_WEIGHT']]
hw['BODY_HEIGHT'] = hw['BODY_HEIGHT'].astype(float)
hw['BODY_WEIGHT'] = hw['BODY_WEIGHT'].astype(float)
hw['BMI'] = hw['BODY_WEIGHT']/(hw['BODY_HEIGHT']/100)**2

In [112]:
bc_cohort_icu = pd.merge(bc_cohort_icu,hw, how = 'left', on = ['PATIENT_ID', 'VISIT_ID'])

In [113]:
bc_cohort_icu.hospstay_seq

0       3.0
1       4.0
2       1.0
3       1.0
4       2.0
       ... 
3886    1.0
3887    2.0
3888    1.0
3889    1.0
3890    1.0
Name: hospstay_seq, Length: 3891, dtype: float64

In [114]:
bc_cohort_icu[bc_cohort_icu['AGE']>=18][['PATIENT_ID','TF_ADMISSION_DATE_TIME']].drop_duplicates().shape

(3537, 2)

In [115]:
bc_cohort_icu['BODY_HEIGHT'] = bc_cohort_icu['BODY_HEIGHT'].apply(lambda x:np.nan if ((x<=100) or (x>=250)) else x)
bc_cohort_icu['BODY_WEIGHT'] = bc_cohort_icu['BODY_WEIGHT'].apply(lambda x:np.nan if ((x<=30) or (x>=230)) else x)
bc_cohort_icu['BMI'] = bc_cohort_icu['BODY_WEIGHT']/(bc_cohort_icu['BODY_HEIGHT']/100)**2

In [116]:
bc_cohort_icu = bc_cohort_icu.drop_duplicates()

In [117]:
#bc_cohort_icu = pd.read_csv('mdro_cohort.csv', encoding = 'ANSI')

In [118]:
bc_cohort_icu.stay_chart_included_mdr.value_counts()

0.0    3114
1.0     423
Name: stay_chart_included_mdr, dtype: int64

In [119]:
pd.isna(bc_cohort_icu).sum().sort_values(ascending =False).head(25)

DOD_HOSP                 3122
SPO2                     2613
firsthospmdrcharttime    2504
firsthospmdrorgname      2504
firsthospmdrRAB          2504
firsthospmdrstoretime    2504
IL_6                      621
glucose                   502
PCT                       252
BMI                       177
BODY_WEIGHT               171
mb                        155
BNP                       134
BODY_HEIGHT               117
mpv                        57
ck_mb                      39
ALP                        24
gamma_GT                   22
lipase                     16
amylase                    13
CRP                         8
mg                          8
chloride                    8
ck                          8
p                           5
dtype: int64

In [120]:
bc_cohort['PCT_nan'] = bc_cohort['PCT'].apply(lambda x:1 if pd.isna(x) else 0)

In [121]:
bc_cohort[bc_cohort['ICU_TYPE_ICU']==1].stay_chart_included_mdr.value_counts()

0.0    3114
1.0     423
Name: stay_chart_included_mdr, dtype: int64

In [122]:
missing_percent = lab_.isnull().mean(axis=1)
lab_filtered = lab_[missing_percent < 0.2]
lab_filtered = lab_filtered[~pd.isna(lab_filtered['wbc'])]

In [123]:
bc_cohort_icu = bc_cohort_icu.merge(lab_filtered[['PATIENT_ID','TF_ADMISSION_DATE_TIME']],how = 'inner', on = ['PATIENT_ID','TF_ADMISSION_DATE_TIME'] )

In [124]:
bc_cohort_icu.shape

(3536, 107)

In [125]:
bc_cohort_icu.drop_duplicates().to_csv('mdro_cohort.csv', index  = False, encoding = 'utf-8')

# 经R语言插补缺失值

In [127]:
bc_cohort_median_fill = pd.read_csv('MDRO_afterimp_Rmice.csv',encoding = 'utf-8')

In [128]:
bc_cohort_median_fill['monocytes_abs'] = bc_cohort_median_fill['monocytes'] * bc_cohort_median_fill['wbc']
bc_cohort_median_fill['neutrophils_abs'] = bc_cohort_median_fill['bands'] * bc_cohort_median_fill['wbc']
bc_cohort_median_fill['eosinophil_abs'] = bc_cohort_median_fill['eosinophil'] * bc_cohort_median_fill['wbc']
bc_cohort_median_fill['basophil_abs'] = bc_cohort_median_fill['basophil'] * bc_cohort_median_fill['wbc']
bc_cohort_median_fill['lymphocytes_abs'] = bc_cohort_median_fill['LYN'] * bc_cohort_median_fill['wbc']
bc_cohort_median_fill['lymphocytes_abs'] = bc_cohort_median_fill['lymphocytes_abs'].apply(lambda x:0.0001 if x == 0 else x)

In [129]:
bc_cohort_median_fill['RPR'] = bc_cohort_median_fill['rdw']/bc_cohort_median_fill['platelet']
bc_cohort_median_fill['NLR'] = bc_cohort_median_fill['neutrophils_abs']/bc_cohort_median_fill['lymphocytes_abs']

bc_cohort_median_fill['SBP'] = bc_cohort_median_fill['SBP'].apply(lambda x:0.0001 if x == 0 else x)
bc_cohort_median_fill['shock_index'] = bc_cohort_median_fill['HR']/bc_cohort_median_fill['SBP']

In [130]:
bc_cohort_median_fill.icustay_rank.value_counts()

1     3199
2      276
3       39
4       11
5        3
6        1
8        1
9        1
14       1
15       1
18       1
19       1
20       1
Name: icustay_rank, dtype: int64

In [132]:
bc_cohort_median_fill.TF_ADMISSION_DATE_TIME = bc_cohort_median_fill.TF_ADMISSION_DATE_TIME.apply(trans_time)

In [133]:
bc_cohort_median_fill.to_csv('plagh_totableone.csv',index = False)

In [134]:
bc_cohort_median_fill['year'] = bc_cohort_median_fill.TF_ADMISSION_DATE_TIME.dt.year

In [135]:
pd.pivot_table(bc_cohort_median_fill, index = ['year'], values = ['stay_chart_included_mdr'], aggfunc= ['count','mean'])

,count,mean
,stay_chart_included_mdr,stay_chart_included_mdr
year,,
2008,77,0.129870
2009,105,0.285714
2010,153,0.222222
2011,148,0.141892
2012,131,0.106870
2013,190,0.157895
2014,412,0.070388
2015,557,0.073609


In [136]:
train_df = bc_cohort_median_fill[bc_cohort_median_fill['year'].isin([2008,2009,2010,2011,2012,2013,2014,2015,2016])]
val_df = bc_cohort_median_fill[bc_cohort_median_fill['year'].isin([2017,2018,2019])]

In [149]:
train_df.to_csv('train_df.csv',index = False)
val_df.to_csv('val_df.csv',index = False)